## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим несколько моделей дерева решений, найдем оптимальную через перекрестную валидацию (CV).

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing
from IPython.display import SVG, display
from graphviz import Source
import os
os.environ["PATH"] += (os.pathsep +
                       'C:/Program Files (x86)/Graphviz2.38/bin/')

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Набор столбцов для расчета
"Облегченный" вариант для визуализации дерева

In [4]:
columns = ["Wt", "Ht", "Ins_Age", "BMI"]

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 1.76 Мб (минус 77.5 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Data columns (total 5 columns):
0           59381 non-null float16
1           59381 non-null float16
2           59381 non-null float16
3           59381 non-null float16
Response    59381 non-null int8
dtypes: float16(4), int8(1)
memory usage: 522.0 KB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3  Response
40140  0.778809  1.003906  0.970703  0.311279         2
24402  0.238281  0.514160 -0.467285  0.000043         7
57279 -0.161133  1.003906 -1.375977 -0.737793         6
27464 -1.594727 -0.955078 -1.073242 -1.542969         8
21574  0.191284  0.514160 -0.315918 -0.055450         8


### Дерево решений
Минимальное число "одинаковых" значений для ветвления - 10

In [9]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model = DecisionTreeClassifier(random_state=0, min_samples_leaf=10)
model.fit(x, data_train["Response"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

### Визуализация дерева
Доступно несколько форматов вывода, большой SVG выводится в Jupyter Notebook сложнее, поэтому используем PNG.

В качестве названий параметров передаем исходный список.

In [10]:
graph = Source(export_graphviz(model, out_file=None,
        feature_names=columns, filled=True,
        class_names=data_train["Response"].unique().astype("str")))
with open("tree.png", "wb") as f:
    f.write(graph.pipe(format="png"))

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.104915 to fit


!["Дерево решений"](tree.png "")

### Влияние признаков
Выведем долю влияния признаков на конечный ответ в дерева

In [11]:
print (model.feature_importances_)

[0.15368974 0.03832401 0.26152987 0.54645638]


### Перекрестная проверка (CV)
Разбиваем обучающую выборку еще на k (часто 5) частей, на каждой части данных обучаем модель. Затем проверяем 1-ю, 2-ю, 3-ю, 4-ю части на 5; 1-ю, 2-ю, 3-ю, 5-ю части на 4 и т.д.

В итоге обучение пройдет весь набор данных, и каждая часть набора будет проверена на всех оставшихся (перекрестным образом).

Перекрестная проверка используется для оптимизации параметров исходной модели - решающего дерева в данном случае. Зададим несколько параметров для перебора и поиска самой точной модели.

Для проверки будем использовать каппа-метрику.

In [12]:
tree_params = {
    'max_depth': range(10, 20),
    'max_features': range(1, round(len(columns_transformed))),
    'min_samples_leaf': range(20, 100)
}

tree_grid = GridSearchCV(model, tree_params, cv=5, n_jobs=2,
            verbose=True, scoring=make_scorer(cohen_kappa_score))
tree_grid.fit(x, data_train["Response"])

Fitting 5 folds for each of 2400 candidates, totalling 12000 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   43.6s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   60.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  1.5min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  2.3min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  3.6min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  4.8min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed:  6.6min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed:  8.2min
[Parallel(n_jobs=2)]: Done 4046 tasks      | elapsed: 10.4min
[Parallel(n_jobs=2)]: Done 4996 tasks      | elapsed: 12.9min
[Parallel(n_jobs=2)]: Done 6046 tasks      | elapsed: 15.5min
[Parallel(n_jobs=2)]: Done 7196 tasks      | elapsed: 18.4min
[Parallel(n_jobs=2)]: Done 8446 tasks      | elapsed: 21.3min
[Parallel(n_jobs=2)]: Done 9796 tasks      | elapsed: 24.4min
[Parallel(n_jobs=2)]: Done 11246 tasks      | elapsed: 27.7mi

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=10,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=2,
             param_grid={'max_depth': range(10, 20),
                         'max_features': range(1, 4),
                      

Выведем самые оптимальные параметры и построим итоговую модель

In [14]:
print (tree_grid.best_params_)
model = DecisionTreeClassifier(random_state=17,
        min_samples_leaf=tree_grid.best_params_['min_samples_leaf'],
        max_features=tree_grid.best_params_['max_features'],
        max_depth=tree_grid.best_params_['max_depth'])
model.fit(x, data_train["Response"])

{'max_depth': 10, 'max_features': 3, 'min_samples_leaf': 97}


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=97, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=17, splitter='best')

### Предсказание данных и оценка модели

In [15]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = model.predict(x_test)

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95

In [17]:
print ("Решающее дерево:",
      round(cohen_kappa_score(data_test["target"],
                data_test["Response"], weights="quadratic"), 3))

Решающее дерево: 0.314


### Матрица неточностей

In [19]:
print ("Решающее дерево:\n", confusion_matrix(data_test["target"],
                data_test["Response"]))

Решающее дерево:
 [[ 176  129    6    2   58  118   73   42]
 [  89  206    8    1   87   54   26   27]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [  72  147   39    0  354  122    7    4]
 [ 245  279   60   46  277  727  401  179]
 [  62   65   10    8   49  144  191   76]
 [ 539  491   90  263  296 1059  920 3553]]
